## Mount elevation telemetry generator

In [ ]:
from lsst.ts import salobj
import asyncio
import os

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

## Go to step 3 if you do not have a mount CSC running

#### 0. set up mount remote

In [ ]:
start_time = datetime.now()
script = salobj.Controller("Script", index=1)
await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too
mount = salobj.Remote(script.domain, "MTMount")
print(f'time to start is {datetime.now() - start_time} [s]')

In [ ]:
await mount.start_task

#### 1. mount status

In [ ]:
sstate = await mount.evt_summaryState.aget(timeout=5)
print('mount state',salobj.State(sstate.summaryState), pd.to_datetime(sstate.private_sndStamp, unit='s'))

In [ ]:
await mount.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.actualPosition, pd.to_datetime(mountAngle.private_sndStamp, unit='s'))

#### 2. Put Mount into STANDBY

In [ ]:
async def checkMountTel(mount):
    print('-------------------------------------------------------')
    for i in range(3):
        await asyncio.sleep(1.0)
        mountAngle = await mount.tel_elevation.aget(timeout=10.)
        print("mount elevation angle", mountAngle.actualPosition, pd.to_datetime(mountAngle.private_sndStamp, unit='s'))

await checkMountTel(mount)
await salobj.set_summary_state(mount, salobj.State.STANDBY)
await checkMountTel(mount)

#### 3. Initiate Mount Controller object

In [ ]:
mountCtrl = salobj.Controller(name="MTMount")
await mountCtrl.start_task

In [ ]:
start_time = Time(datetime.now())
freq = 1 #Hz
end_time = start_time + timedelta(minutes=80)
while Time(datetime.now())< end_time:
    await asyncio.sleep(1.0/freq)
    #mountCtrl.tel_elevation.set_put(angleActual=mountAngle.angleActual+dAngle)
    #mountCtrl.tel_elevation.set_put(angleActual=89.5)
    mountCtrl.tel_elevation.set_put(actualPosition=40)

### Close up

In [ ]:
await mountCtrl.close()

In [ ]:
await salobj.set_summary_state(mount, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(mount, salobj.State.ENABLED)